<a href="https://colab.research.google.com/github/rikasyahriani/MIDTERMRIKA/blob/main/MIDTERMRIKA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama : Rika Syahriani
NPM : 2206130782

In [ ]:
# Clone the YOLOv5 repository (contains the YOLOv5 model code)
!git clone https://github.com/ultralytics/yolov5
%cd yolov5

# Install the required dependencies for YOLOv5
!pip install -r requirements.txt

# Install OpenCV for video processing
!pip install opencv-python-headless

# Install PyTorch with CUDA support (ensures GPU acceleration)
!pip install torch torchvision torchaudio

# Check if GPU is available (optional, to confirm GPU is enabled in Colab)
import torch
print(torch.cuda.is_available())



Cloning into 'yolov5'...
remote: Enumerating objects: 17022, done.
remote: Total 17022 (delta 0), reused 0 (delta 0), pack-reused 17022 (from 1)
Receiving objects: 100% (17022/17022), 15.62 MiB | 12.71 MiB/s, done.
Resolving deltas: 100% (11694/11694), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.5/877.5 kB 14.7 MB/s eta 0:00:00
False


In [ ]:
from google.colab import files

# This will prompt you to upload a video file from your local system
uploaded = files.upload()

# To confirm the file is uploaded and check its name
for filename in uploaded.keys():
    print(f'Uploaded file: {filename}')


Saving inihaechan.mp4 to inihaechan.mp4
Uploaded file: inihaechan.mp4


In [ ]:
import cv2
import torch

# Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)  # Load a pre-trained YOLOv5 model

# Define the video path
video_path = 'inihaechan.mp4'  # Replace with your video path

/usr/local/lib/python3.10/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2024-10-25 Python-3.10.12 torch-2.5.0+cu121 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 108MB/s] 

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [ ]:
from google.colab.patches import cv2_imshow
import cv2
import torch

# Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)  # Load a pre-trained YOLOv5 model

# Define the video path
video_path = 'inihaechan.mp4'  # Replace with your video path

# Open the video file
cap = cv2.VideoCapture(video_path)

# Process each frame
frame_count = 0
max_frames = 100  # Set a limit for frames to process

while cap.isOpened() and frame_count < max_frames:
    ret, frame = cap.read()
    if not ret:
        break

    # Perform inference on the frame
    results = model(frame)

    # Extract bounding boxes and class labels
    bbox_xyxy = results.xyxy[0]  # Bounding boxes
    for *bbox, conf, cls in bbox_xyxy:  # bbox format: [x1, y1, x2, y2, confidence, class]
        x1, y1, x2, y2 = map(int, bbox)  # Convert to integers
        label = model.names[int(cls)]  # Get class label

        # Draw bounding box on the frame
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(frame, f'{label} {conf:.2f}', (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Display the frame using cv2_imshow
    cv2_imshow(frame)

    frame_count += 1

# Release the video capture object
cap.release()

In [ ]:
# Store detection results
detections = []

# Process each frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    bbox_xyxy = results.xyxy[0]

    # Preprocess the results
    for *bbox, conf, cls in bbox_xyxy:
        x1, y1, x2, y2 = map(int, bbox)
        label = model.names[int(cls)]
        detection = {
            'frame': frame_number,  # frame number
            'bounding_box': [x1, y1, x2, y2],
            'confidence': conf.item(),
            'class_label': label
        }
        detections.append(detection)

    frame_number += 1

# Save the detections for analysis
import pandas as pd
detections_df = pd.DataFrame(detections)
detections_df.to_csv('inihaechan.mp4', index=False)

In [ ]:
# Simple tracking logic (not using a dedicated tracking library)
tracked_objects = {}

# Iterate through the detections
for detection in detections:
    frame_number = detection['frame']
    bbox = detection['bounding_box']
    label = detection['class_label']

    # Maintain object identity
    if label not in tracked_objects:
        tracked_objects[label] = []

    tracked_objects[label].append({
        'frame': frame_number,
        'bounding_box': bbox
    })

# Example of accessing tracking data
for obj, tracks in tracked_objects.items():
    print(f'Tracking {obj}: {len(tracks)} frames')

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True, force_reload=True)
model.eval()  # Set to evaluation mode

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-10-25 Python-3.10.12 torch-2.5.0+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  